In [ ]:
!wget http://dept-info.labri.fr/~hanna/Pub/features_adapte.csv
!wget http://dept-info.labri.fr/~hanna/Pub/features_head.csv
# !wget http://dept-info.labri.fr/~hanna/Pub/train_clean.csv
# !wget http://dept-info.labri.fr/~hanna/Pub/test_clean.csv

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [2]:
# Croisement features/tracks du dataset train

# Nom des features
features = pd.read_csv(filepath_or_buffer="features_head.csv", sep=",")
print(features.columns)
#print(features)
print("#################")

# Croisement features/tracks du dataset train
traingenre = pd.read_csv(filepath_or_buffer="train_clean.csv", sep=",")
iter_csv = pd.read_csv(filepath_or_buffer="features_adapte.csv", sep=",", iterator=True, chunksize=10000)
datatrain = pd.concat([chunk for chunk in iter_csv])

data = pd.merge(traingenre, datatrain, on='track_id')
data.sample(n=10)

Index(['feature', 'chroma_cens', 'chroma_cens.1', 'chroma_cens.2',
       'chroma_cens.3', 'chroma_cens.4', 'chroma_cens.5', 'chroma_cens.6',
       'chroma_cens.7', 'chroma_cens.8',
       ...
       'tonnetz.39', 'tonnetz.40', 'tonnetz.41', 'zcr', 'zcr.1', 'zcr.2',
       'zcr.3', 'zcr.4', 'zcr.5', 'zcr.6'],
      dtype='object', length=519)
#################


,track_id,genre_id,01,02,03,04,05,06,07,08,...,04.41,05.41,06.41,01.70,01.71,01.72,01.73,01.74,01.75,01.76
54,1682,5,-0.759522,-0.660455,-0.007256,-0.116650,-0.829314,-0.310002,-1.076483,-0.993827,...,0.115826,0.023135,0.022315,70.803398,0.741699,0.052991,0.046387,0.000000,6.811990,0.043964
2879,110761,4,-1.127796,-0.967607,0.212393,-0.962679,-0.652817,-0.841560,-0.425706,-0.806104,...,0.079879,0.024087,0.020425,5.799253,0.526855,0.055394,0.028320,0.000000,2.233683,0.070390
3804,145209,1,-0.790500,-0.611869,-0.613862,-0.269213,-0.463088,-0.523348,-0.413741,-0.925254,...,0.099218,0.022588,0.022188,1.715044,0.153809,0.041892,0.037109,0.002441,1.224427,0.022510
1475,55099,4,1.282881,0.631721,1.528502,-0.042285,0.054985,0.878764,0.775658,0.212126,...,0.057156,0.016998,0.018194,4.588485,0.469238,0.064075,0.034668,0.001953,2.009240,0.072844
25,821,7,-0.945239,1.634213,0.459780,1.814182,-0.651956,-0.775463,-0.404574,1.973060,...,0.140010,0.024466,0.027042,78.337555,0.763672,0.030665,0.014648,0.001465,7.906507,0.063550
2784,108059,6,-0.388243,-0.149367,0.007961,0.163642,-0.348244,0.013897,-0.704634,-0.722521,...,0.086807,0.021923,0.019604,8.106359,0.232422,0.056917,0.050781,0.000000,2.209893,0.031030
3353,127191,3,3.499011,-1.375531,-1.620836,-1.472571,3.564409,-0.680405,-1.222306,-0.844869,...,0.179666,0.029854,0.027233,32.134811,0.753418,0.041782,0.017090,0.001465,5.600079,0.106448
3174,121276,6,-0.298821,-0.791905,-0.577557,-0.290424,-0.103356,0.647770,-0.437137,-0.564280,...,0.080147,0.025150,0.025666,1.218696,0.214355,0.053546,0.047363,0.000000,0.937745,0.027033
342,12355,3,0.035939,-0.537234,-0.463681,-1.012958,0.085915,-0.734578,-0.789860,-0.793055,...,0.126793,0.028062,0.023247,39.044254,0.537598,0.038208,0.032227,0.000000,4.653836,0.028483
1762,63802,4,0.121118,-0.305045,-0.334666,0.751004,-1.091425,-1.204226,-1.108748,0.524335,...,0.050565,0.024549,0.014623,11.127469,0.403809,0.044436,0.037109,0.002441,2.244269,0.030539


In [3]:
# training sets
x = data.drop(['genre_id', 'track_id'], axis=1)
y = data['genre_id'].values

print("x :", x.shape, ", y :", y.shape)

x : (3995, 518) , y : (3995,)


In [4]:
# split train test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

print("x_train :", x_train.shape, ", y_train :", y_train.shape)
print("x_test :", x_test.shape, ", y_test :", y_test.shape)

x_train : (3196, 518) , y_train : (3196,)
x_test : (799, 518) , y_test : (799,)


In [5]:
# normalisation
scaler = preprocessing.StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

mean_train = x_train.mean()
std_train = x_train.std()
print(f'mean_train: {mean_train}. std_train: {std_train}')

mean_train: 3.4462134719471516e-17. std_train: 0.9951620422561797


## Premier modèle : K nearest neighbors

In [6]:
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [7]:
y_pred = knn_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.82      0.84      0.83        95
           2       0.48      0.19      0.27       109
           3       0.50      0.61      0.55        89
           4       0.44      0.43      0.43       101
           5       0.36      0.59      0.45        99
           6       0.47      0.22      0.30       116
           7       0.30      0.36      0.33       107
           8       0.39      0.52      0.44        83

    accuracy                           0.45       799
   macro avg       0.47      0.47      0.45       799
weighted avg       0.47      0.45      0.44       799



## Deuxième modèle : Decision Tree

In [8]:
tree_model = DecisionTreeClassifier(max_leaf_nodes=16, min_samples_split=2)
tree_model.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=16,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [9]:
y_pred = tree_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.76      0.55      0.64        95
           2       0.38      0.18      0.25       109
           3       0.42      0.49      0.45        89
           4       0.44      0.38      0.41       101
           5       0.30      0.26      0.28        99
           6       0.28      0.34      0.30       116
           7       0.19      0.32      0.24       107
           8       0.45      0.45      0.45        83

    accuracy                           0.36       799
   macro avg       0.40      0.37      0.38       799
weighted avg       0.39      0.36      0.37       799



In [10]:
# Attention !!! Prend beaucoup de temps d'execution. Valeurs trouvée déjà utilisées précédemment.
# Trouvé : max_leaf_nodes = 16 et min_samples_split = 2

# optimization
# parameters = {'max_leaf_nodes':[x for x in range(2, 20)], 'min_samples_split':[y for y in range(2, 20)]} 

# grid_search_cv = GridSearchCV(DecisionTreeClassifier(), parameters, scoring='accuracy')
# grid_search_cv.fit(x_train, y_train)
# grid_search_cv.best_estimator_

In [11]:
# y_pred = grid_search_cv.predict(x_test)
# print(classification_report(y_test,y_pred))

## Troisième modèle : Random Forest

In [12]:
forest_model = RandomForestClassifier(max_leaf_nodes=16, min_samples_split=2)
forest_model.fit(x_train, y_train)

/home/janerussel/anaconda3/envs/IA/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=16,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
y_pred = forest_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.61      0.80      0.69        95
           2       0.44      0.29      0.35       109
           3       0.44      0.65      0.52        89
           4       0.39      0.62      0.48       101
           5       0.39      0.43      0.41        99
           6       0.63      0.21      0.31       116
           7       0.43      0.06      0.10       107
           8       0.40      0.70      0.51        83

    accuracy                           0.45       799
   macro avg       0.47      0.47      0.42       799
weighted avg       0.47      0.45      0.41       799



## Quatrième méthode : Gradient Boosting

In [14]:
gradient_model = GradientBoostingClassifier()
gradient_model.fit(x_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [15]:
y_pred = gradient_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.83      0.84      0.84        95
           2       0.50      0.47      0.48       109
           3       0.59      0.69      0.64        89
           4       0.62      0.62      0.62       101
           5       0.66      0.58      0.62        99
           6       0.57      0.46      0.51       116
           7       0.33      0.37      0.35       107
           8       0.56      0.66      0.61        83

    accuracy                           0.58       799
   macro avg       0.58      0.59      0.58       799
weighted avg       0.58      0.58      0.58       799



## Cinquième méthode : XGBoost

In [16]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [17]:
y_pred = xgb_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.84      0.89      0.87        95
           2       0.48      0.50      0.49       109
           3       0.63      0.70      0.66        89
           4       0.60      0.63      0.62       101
           5       0.59      0.52      0.55        99
           6       0.56      0.42      0.48       116
           7       0.36      0.35      0.35       107
           8       0.50      0.64      0.56        83

    accuracy                           0.57       799
   macro avg       0.57      0.58      0.57       799
weighted avg       0.57      0.57      0.57       799



## Tuning XGBoost

In [18]:
# params = {
#     # Parameters that we are going to tune.
#     'max_depth':6,
#     'min_child_weight': 1,
#     'eta':.3
# }

# params['eval_metric'] = "mae"
# num_boost_round = 999

# model = xgb.train(
#     params,
#     x_train,
#     num_boost_round=num_boost_round,
#     evals=[(y_train, "Test")],
#     early_stopping_rounds=10
# )

# xgb_model = xgb.XGBClassifier()
# xgb_model.fit(x_train, y_train)

In [19]:
# parameters = {'n_estimators':[n for n in range(100, 200, 10)], 'learning_rate':[x/100 for x in range(5, 30)], 'max_depth':[y for y in range(3, 10)], 'min_child_weight':[z for z in range(1, 6)]} 
# grid_search_cv = GridSearchCV(xgb.XGBClassifier(), parameters, scoring='accuracy')
# grid_search_cv.fit(x_train, y_train)
# grid_search_cv.best_estimator_

In [20]:
xgb_model = xgb.XGBClassifier(n_estimators=200, learning_rate=0.2, max_depth=3, min_child_weight=1)
xgb_model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [21]:
y_pred = xgb_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.88      0.88      0.88        95
           2       0.49      0.51      0.50       109
           3       0.61      0.75      0.68        89
           4       0.65      0.61      0.63       101
           5       0.63      0.48      0.55        99
           6       0.59      0.47      0.52       116
           7       0.37      0.38      0.37       107
           8       0.52      0.65      0.58        83

    accuracy                           0.58       799
   macro avg       0.59      0.59      0.59       799
weighted avg       0.59      0.58      0.58       799



In [22]:
# n_estimators=150, learning_rate=0.2, max_depth=3, min_child_weight=1  --------> 0.58
# n_estimators=150, learning_rate=0.2, max_depth=5, min_child_weight=1  --------> 0.58
# n_estimators=180, learning_rate=0.2, max_depth=5, min_child_weight=3  --------> 0.59
# n_estimators=180, learning_rate=0.25, max_depth=5, min_child_weight=3  -------> 0.58
# n_estimators=170, learning_rate=0.2, max_depth=7, min_child_weight=3  --------> 0.58
# n_estimators=180, learning_rate=0.15, max_depth=5, min_child_weight=3  -------> 0.58
# n_estimators=200, learning_rate=0.1, max_depth=5, min_child_weight=3  --------> 0.58
# n_estimators=200, learning_rate=0.2, max_depth=3, min_child_weight=1  --------> 0.59